In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random

In [5]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.aruodas.lt/butai/puslapis/1/?FOwnerDbId0=1&FOwnerDbId2=1'
driver.get(url)
time.sleep(5)
source = driver.page_source


page = 1
# page = 149
# while page < 201:
while page < 3:
    source = driver.page_source
    
    bs = BeautifulSoup(source, 'html.parser')

    butai_nuorodos = []

    linkai = bs.find_all('h3')
    for linkas in linkai:
        l = linkas.find('a')
        if l != None:
            butai_nuorodos.append(l['href'])

    kitas = bs.find('div', {'class':'pagination'}).find_all('a', {'class':'page-bt'})
    for k in kitas:
        # print(k.text.strip())
        if k.text.strip() == '»':
            kitas_psl = k['href']
            # print(kitas_psl)
    next_nuoroda = f'https://www.aruodas.lt/{kitas_psl}'
    
    for nuoroda in butai_nuorodos:
        # print(nuoroda)
        
        duomenys = {
            'miestas':'None',
            'rajonas':'None',
            'gatve':'None',
            'kaina':'None',
            'kv_kaina':'None',
            'Plotas:':'None',
            'Kambarių sk.:':'None',
            'Metai:':'None',
            'Aukštas:':'None',
            'Aukštų sk.:':'None',
            'Namo numeris:':'None',
            'Buto numeris:':'None',
            'Pastato tipas:':'None',
            'Šildymas:':'None',
            'Įrengimas:':'None',
            'Pastato energijos suvartojimo klasė:':'None',
            'Ypatybės:':'None',
            'Papildomos patalpos:':'None',
            'Papildoma įranga:':'None',
            'Apsauga:':'None',
            'nuoroda':'None'
        }

        duomenys['nuoroda'] = nuoroda
        
        url = nuoroda
        driver.get(url)
        time.sleep(2)
        source = driver.page_source

        bs_buto = BeautifulSoup(source, 'html.parser')

        gyvenviete = bs_buto.find('h1', {'class':'obj-header-text'}).text
        gyvenviete_lst = gyvenviete.split(',')
        if len(gyvenviete_lst) == 4:
            miestas = gyvenviete_lst[0].strip()
            duomenys['miestas'] = miestas
            # print(miestas)
            rajonas = gyvenviete_lst[1].strip()
            duomenys['rajonas'] = rajonas
            # print(rajonas)
            gatve = gyvenviete_lst[2].strip()
            duomenys['gatve'] = gatve
            # print(gatve)
        if len(gyvenviete_lst) == 3:
            miestas = gyvenviete_lst[0].strip()
            duomenys['miestas'] = miestas
            # print(miestas)
            gatve = gyvenviete_lst[1].strip()
            duomenys['gatve'] = gatve

            
        kaina = int(bs_buto.find('span', {'class':'price-eur'}).text.strip().replace('€', '').replace(' ', ''))
        duomenys['kaina'] = kaina
        # print(kaina)

        kv_kaina_txt = bs_buto.find('span', {'class':'price-per'}).text.strip()
        # print(kv_kaina_txt)
        kv_kaina = int(kv_kaina_txt.split('€')[0].replace(' ', '').replace('(', ''))
        duomenys['kv_kaina'] = kv_kaina
        # print(kv_kaina)


        info_keys = []
        info_values = []
        info_k = bs_buto.find('dl', {'class':'obj-details'}).find_all('dt')
        for i in info_k:
            # print(i.text.strip())
            info_keys.append(i.text.strip())
            
        # print(info_keys)

        info_v = bs_buto.find('dl', {'class':'obj-details'}).find_all('dd')
        for i in info_v:
            # print(i.text.strip())
            info_values.append(i.text.strip())
        # print(info_values)

        for k, v in zip(info_keys, info_values):
            duomenys[k] = v

        print(duomenys)
        
        data = tuple(duomenys.values())
        data1 = []
        data1.append(data)
        
        SDB = sqlite3.connect('Aruodas_test.db')  # jei neegzistuoja db , bus sukurta nauja db
        Cs = SDB.cursor()
        
        #  if not exists - tikrina ar jau sukurta DB
        sql = '''create table if not exists Aruodas
        (
        miestas text not null,
        rajonas text not null,
        gatve text not null,
        kaina numeric not null,
        kvadatoKaina numeric not null,
        plotas text not null,
        kambariai text not null,
        metai text not null,
        aukstas text not null,
        aukstuSk text not null,
        namoNr text not null,
        butoNr text not null,
        pastatoTipas text not null,
        sildymas text not null,
        irengimas text not null,
        energetineKlase text not null,
        ypatybes text not null,
        papildomosPatalpos text not null,
        papildomaIranga text not null,
        apsauga text not null,
        nuoroda text not null
        )
        '''
        Cs.execute(sql)

        sql_template = '''insert into Aruodas values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
        Cs.executemany(sql_template, data1)

        SDB.commit() # commit() butinas jei norim irasyti duomenis i DB

        SDB.close()
        print('data written to DB')
        
        
    driver.get(next_nuoroda)
    
    # su get metodu atsidarom next_nuoroda
    # nuskaitome next puslapi
    # pasiimam nuajas auto nuorodas ir nauja next nuorodas
    # cikla kartojam is naujo
    page = page +1
    print('data written to DB')
    print(f'kitas puslapis: {kitas_psl}')
    
    
driver.close()
print('Baigta!!!')

{'miestas': 'Vilnius', 'rajonas': 'Žirmūnai', 'gatve': 'Riterių g.', 'kaina': 165000, 'kv_kaina': 4072, 'Plotas:': '40,52 m²', 'Kambarių sk.:': '2', 'Metai:': '2023', 'Aukštas:': '5', 'Aukštų sk.:': '8', 'Namo numeris:': '10', 'Buto numeris:': '127', 'Pastato tipas:': 'Blokinis', 'Šildymas:': 'Centrinis kolektorinis', 'Įrengimas:': 'Įrengtas', 'Pastato energijos suvartojimo klasė:': 'A', 'Ypatybės:': 'None', 'Papildomos patalpos:': 'None', 'Papildoma įranga:': 'None', 'Apsauga:': 'None', 'nuoroda': 'https://www.aruodas.lt/butai-vilniuje-zirmunuose-riteriu-g-tik-ka-naujai-irengtas-kambariu-butas-1-3472873/'}
data written to DB
{'miestas': 'Palanga', 'rajonas': 'Palanga', 'gatve': 'A. Mickevičiaus g.', 'kaina': 268000, 'kv_kaina': 7657, 'Plotas:': '35 m²', 'Kambarių sk.:': '2', 'Metai:': '2023', 'Aukštas:': '1', 'Aukštų sk.:': '3', 'Namo numeris:': 'None', 'Buto numeris:': 'None', 'Pastato tipas:': 'Kita', 'Šildymas:': 'Centrinis kolektorinis', 'Įrengimas:': 'Įrengtas', 'Pastato energijo